In [1]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import functions as fn
import warnings
warnings.filterwarnings('ignore')

In [3]:
df_CCM = pd.read_csv('data/ccm_manipulated_347418.csv', names=['EVENT_ID', 'SEQUENCE_ID', 'R_OS_ID', 'PRODUCTION_LINE_NAME', 'REFERENCE_DATE', 'PIECE_ID', 
 'MATERIAL_ID', 'MOLD_WIDTH', 'WIDTH', 'THICKNESS', 'WEIGHT', 'LENGTH', 'HEAT_ID',
 'STEEL_GRADE_ID_INT', 'EXIT_TEMP', 'SLAB_TRANSITION'], header=None)
df_CCM

,EVENT_ID,SEQUENCE_ID,R_OS_ID,PRODUCTION_LINE_NAME,REFERENCE_DATE,PIECE_ID,MATERIAL_ID,MOLD_WIDTH,WIDTH,THICKNESS,WEIGHT,LENGTH,HEAT_ID,STEEL_GRADE_ID_INT,EXIT_TEMP,SLAB_TRANSITION
0,8360,109,115587,CCM1,13.12.16,16000041-07,148513,#NAME?,1330.0,87,2.140000e+09,2.440000e+09,16000041.0,26.0,0.0,0
1,8359,109,115587,CCM1,13.12.16,16000041-06,148512,#NAME?,1330.0,87,2.140000e+09,2.440000e+09,16000041.0,26.0,0.0,0
2,8358,109,115587,CCM1,13.12.16,16000041-05,148511,#NAME?,1330.0,87,2.140000e+09,2.440000e+09,16000041.0,26.0,0.0,0
3,8357,109,115587,CCM1,13.12.16,16000041-03,148509,#NAME?,1330.0,87,2.140000e+09,2.440000e+09,16000041.0,26.0,0.0,0
4,8356,109,115587,CCM1,13.12.16,16000041-02,148508,#NAME?,1330.0,87,2.140000e+09,2.440000e+09,16000041.0,26.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
347413,458481,4154,891748,CCM1,13.02.18,18023901-07,352574,#NAME?,1530.0,67,1.400000e+09,1.830000e+09,18023901,50.0,0.0,0
347414,458480,4154,891748,CCM1,13.02.18,18023901-06,352573,#NAME?,1530.0,67,1.400000e+09,1.830000e+09,18023901,50.0,0.0,0
347415,458479,4154,891748,CCM1,13.02.18,18023901-05,352572,#NAME?,1530.0,67,1.400000e+09,1.830000e+09,18023901,50.0,0.0,0
347416,458478,4154,891748,CCM1,13.02.18,18023901-04,352571,#NAME?,1530.0,67,1.400000e+09,1.830000e+09,18023901,50.0,0.0,0
